In [18]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sys
import os
sys.path.append('./../dataProcessing/')
from dataProcessing import GetData
from sklearn.model_selection import train_test_split
import configparser
from os import path
import sys
# sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))
import datetime as dt


import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.model_selection import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score

## for Deep-learing:
import keras
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout



# hard code
paths = ['/Users/phamduy/github-res/stock-info/dataProcessing/Data/ext/01_01_2015_31_08_2019']
instance = GetData()
input_data = instance.read_excel(paths)

/Users/phamduy/github-res/stock-info/dataProcessing/Data/ext/01_01_2015_31_08_2019/UPCOM.xls
/Users/phamduy/github-res/stock-info/dataProcessing/Data/ext/01_01_2015_31_08_2019/Index.xls
/Users/phamduy/github-res/stock-info/dataProcessing/Data/ext/01_01_2015_31_08_2019/HOSE.xls
/Users/phamduy/github-res/stock-info/dataProcessing/Data/ext/01_01_2015_31_08_2019/HNX.xls


In [19]:
dataset = input_data.get('index')
dataset.describe()
dataset=dataset.drop(["OPEN","HIGH","LOW"],axis=1)
# dataset['DATE'] = pd.to_datetime(dataset['DATE'])
# dataset['DATE']=dataset['DATE'].map(dt.datetime.toordinal)

In [20]:
vnindex = dataset.loc[dataset.TICKER=='VNINDEX',: ]
upcomindex = dataset.loc[dataset.TICKER=='UPCOMINDEX',: ]

vnindex["VOLUME"] = pd.to_numeric(vnindex["VOLUME"])
y = vnindex.pop('CLOSE')
X = vnindex.drop(['TICKER'],axis=1)
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.3, random_state=0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [33]:
y_train = np.array(y_train).reshape(len(y_train), 1)
X_train = np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1)

In [34]:
print(X_train.shape)
print(y_train.shape)

(782, 2, 1)
(782, 1)


In [36]:
X_train

array([[[Timestamp('2015-03-06 00:00:00')],
        [101676233]],

       [[Timestamp('2015-08-07 00:00:00')],
        [87763805]],

       [[Timestamp('2015-08-28 00:00:00')],
        [165727370]],

       ...,

       [[Timestamp('2018-07-04 00:00:00')],
        [128079017]],

       [[Timestamp('2017-05-23 00:00:00')],
        [0]],

       [[Timestamp('2017-11-16 00:00:00')],
        [0]]], dtype=object)

In [35]:
# Model building
length_of_sequence = X_train.shape[1]
in_out_neurons = 1
n_hidden = 300
 
model = Sequential()
model.add(LSTM(n_hidden, batch_input_shape=(None, length_of_sequence, in_out_neurons), return_sequences=False))
model.add(Dense(in_out_neurons))
model.add(Activation("linear"))
optimizer = Adam(lr=0.001)
model.compile(loss="mean_squared_error", optimizer=optimizer)
model.summary()
# Learning 
history = model.fit(X_train, y_train, batch_size=1, epochs=100, validation_split=0.1)
predicted = model.predict(X_test)
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 300)               362400    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 301       
_________________________________________________________________
activation_7 (Activation)    (None, 1)                 0         
Total params: 362,701
Trainable params: 362,701
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 703 samples, validate on 79 samples
Epoch 1/100


TypeError: float() argument must be a string or a number, not 'Timestamp'